In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_auc_score

# 1. 파일 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'
OOF_PATH = './oof_data/'

M1_CSV = '0130-1923.csv'
M1_OOF = 'exp11_sota_AUC_0.77212.npy'
EXP40_CSV = '40_Pure_PL_5pct_AUC_0.77376.csv'
EXP40_OOF = 'exp40_pure_pl_5pct_AUC_0.77376.npy'

# 2. 각 모델별 마스크 및 인덱스 생성
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
a_cols = [col for col in train_df.columns if col.endswith('A') and col.startswith('Q')]

# m1용 마스크 (45,529행)
m1_mask = (train_df['familysize'] <= 50)
m1_indices = train_df[m1_mask].index

# m7/Exp40용 마스크 (45,490행)
m7_mask = m1_mask & (train_df[a_cols].std(axis=1) > 0)
m7_indices = train_df[m7_mask].index

# 3. 로컬 AUC 시뮬레이션 (리인덱싱으로 39행 차이 해결)
oof_m1_raw = np.load(os.path.join(OOF_PATH, M1_OOF)).flatten()
# m1 OOF를 원래 인덱스(45,529)에 매핑한 후, m7 인덱스(45,490)로 리인덱싱하여 교집합 추출
oof_m1 = pd.Series(oof_m1_raw, index=m1_indices).reindex(m7_indices)

oof_40_raw = np.load(os.path.join(OOF_PATH, EXP40_OOF)).flatten()
# Exp 40은 m7 인덱스 기반으로 생성되었으므로 앞부분만 매핑
oof_40 = pd.Series(oof_40_raw[:len(m7_indices)], index=m7_indices)

train_y = (2 - train_df.loc[m7_indices, 'voted'].to_numpy()).astype(np.float32)

# 랭크 변환 및 시뮬레이션
rank_m1_oof = oof_m1.rank(pct=True)
rank_40_oof = oof_40.rank(pct=True)
sim_auc = roc_auc_score(train_y, (rank_m1_oof * 0.5) + (rank_40_oof * 0.5))

print('-' * 50)
print(f'** Exp 44 로컬 검증 결과 (인덱스 보정 완료) **')
print(f'm1 rows: {len(oof_m1)} | Exp40 rows: {len(oof_40)}')
print(f'Exp 26 로컬 기준: 0.77364')
print(f'Exp 44 (5:5) 예측 AUC: {sim_auc:.5f}')
print('-' * 50)

# 4. 최종 파일 생성
if sim_auc > 0.77364:
    m1_sub = pd.read_csv(os.path.join(SUB_PATH, M1_CSV))
    exp40_sub = pd.read_csv(os.path.join(SUB_PATH, EXP40_CSV))
    sample_sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

    # 리인덱싱 없이 rank를 바로 써도 되는 이유는 CSV 파일들이 이미 동일한 test_x 순서를 따르기 때문입니다.
    rank_m1_sub = m1_sub['voted'].rank(pct=True)
    rank_40_sub = exp40_sub['voted'].rank(pct=True)
    final_rank = (rank_m1_sub * 0.5) + (rank_40_sub * 0.5)

    submission = sample_sub.copy()
    submission['voted'] = m1_sub['voted'].sort_values().values[
        (final_rank * (len(final_rank) - 1)).astype(int)
    ]

    output_name = os.path.join(SUB_PATH, '44_Final_Master_Jumper_55.csv')
    submission.to_csv(output_name, index=False)
    print(f'최종 파일 생성 완료: {output_name}')

--------------------------------------------------
** Exp 44 로컬 검증 결과 (인덱스 보정 완료) **
m1 rows: 45490 | Exp40 rows: 45490
Exp 26 로컬 기준: 0.77364
Exp 44 (5:5) 예측 AUC: 0.77362
--------------------------------------------------
